In [210]:
import pandas as pd

In [211]:
data = pd.read_csv("freshmart_products_csv.csv")



In [212]:
data.head()

,ProductID,ProductName,Category,Price,StockQuantity
0,1,Milk 1,Dairy,19.35,358.0
1,2,Cola 2,Beverages,9.99,81.0
2,3,Banana 3,Produce,30.33,112.0
3,4,Cream 4,Dairy,22.85,224.0
4,5,Cream 5,Dairy,3.85,97.0


In [213]:
data.shape


(205, 5)

In [214]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ProductID      205 non-null    int64  
 1   ProductName    205 non-null    object 
 2   Category       205 non-null    object 
 3   Price          200 non-null    float64
 4   StockQuantity  200 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 8.1+ KB


In [215]:
data.isnull().sum() # have a preview of the null values in our dataset


ProductID        0
ProductName      0
Category         0
Price            5
StockQuantity    5
dtype: int64

In [216]:
# Drop rows with any null values
data = data.dropna()

In [217]:
data.isnull().sum() # have a preview of the null values in our dataset


ProductID        0
ProductName      0
Category         0
Price            0
StockQuantity    0
dtype: int64

In [218]:
print(data.duplicated())  # Returns True for duplicate rows

0      False
1      False
2      False
3      False
4      False
       ...  
200     True
201     True
202    False
203     True
204     True
Length: 195, dtype: bool


In [219]:
data.duplicated().sum() # Count duplicate rows

4

In [220]:
data = data.drop_duplicates()

In [221]:
data.duplicated().sum() 

0

In [222]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191 entries, 0 to 202
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ProductID      191 non-null    int64  
 1   ProductName    191 non-null    object 
 2   Category       191 non-null    object 
 3   Price          191 non-null    float64
 4   StockQuantity  191 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 9.0+ KB


In [223]:
data["StockValue"] = data["Price"] * data["StockQuantity"]

In [224]:
data.head()

,ProductID,ProductName,Category,Price,StockQuantity,StockValue
0,1,Milk 1,Dairy,19.35,358.0,6927.30
1,2,Cola 2,Beverages,9.99,81.0,809.19
2,3,Banana 3,Produce,30.33,112.0,3396.96
3,4,Cream 4,Dairy,22.85,224.0,5118.40
4,5,Cream 5,Dairy,3.85,97.0,373.45


In [225]:
# Aggregate the data
result =data.groupby("Category").agg(AvgPrice=("Price", "mean"),TotalStock=("StockQuantity", "sum"))
print(result)

            AvgPrice  TotalStock
Category                        
Bakery     22.788286      9699.0
Bevarages   9.360000       176.0
Beverages  25.427179     10815.0
Dairy      24.570000      9240.0
Produce    25.148684      9221.0
Snacks     25.231892     10673.0
dairy       9.990000       469.0
snakcs     21.830000        63.0


In [226]:
import psycopg2

# Connection parameters
host = "localhost"      # or your server IP
dbname = "Freshmart"
user = "postgres"
password = "sheymoore2005"
port = "5432"           # default PostgreSQL port

try:
    # Establish connection
    conn = psycopg2.connect(
        host=host,
        database=dbname,
        user=user,
        password=password,
        port=port
    )
    
    # Create a cursor
    cursor = conn.cursor()
    
    print("✅ Connection successful!")
    
    # Example query
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print("Database version:", db_version)
    
    # Close cursor and connection
    #cursor.close()
   # conn.close()

except Exception as e:
    print("❌ Error connecting to database:", e)


✅ Connection successful!
Database version: ('PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit',)


In [227]:
cursor.execute("""
        CREATE TABLE IF NOT EXISTS freshmart_products (
            ProductID INT PRIMARY KEY,
            ProductName TEXT NOT NULL,
            Category TEXT NOT NULL,
            Price NUMERIC(10,2) NOT NULL,
            StockQuantity INT NOT NULL
        );
    """)
conn.commit()

    # Insert rows from DataFrame
for _, row in data.iterrows():
    cursor.execute("""
    INSERT INTO freshmart_products (ProductID, ProductName, Category, Price, StockQuantity)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (ProductID) DO UPDATE
    SET ProductName = EXCLUDED.ProductName,
    Category = EXCLUDED.Category,
    Price = EXCLUDED.Price,
    StockQuantity = EXCLUDED.StockQuantity;
    """, (int(row["ProductID"]), row["ProductName"], row["Category"], float(row["Price"]), int(row["StockQuantity"])))
conn.commit()
print("✅ ETL load successful!")

✅ ETL load successful!


In [228]:
# 1. All products in the "Dairy" category
query_dairy ="SELECT * FROM freshmart_products WHERE Category = 'Dairy';"
data_dairy = pd.read_sql(query_dairy, conn)
print("All Dairy Products:")
print(data_dairy.to_string(index=False))  # clean display

# 2. Products with StockQuantity < 50
query_low_stock = "SELECT * FROM freshmart_products WHERE StockQuantity < 50;"
data_low_stock = pd.read_sql(query_low_stock, conn)
print("\n Products with StockQuantity < 50:")
print(data_low_stock.to_string(index=False))


conn.close()

All Dairy Products:
 productid productname category  price  stockquantity
         1      Milk 1    Dairy  19.35            358
        70   Yogurt 70    Dairy  38.02             45
        80   Yogurt 80    Dairy   6.39            386
         4     Cream 4    Dairy  22.85            224
         5     Cream 5    Dairy   3.85             97
        10    Cream 10    Dairy   1.04            453
        19   Butter 19    Dairy  49.18            140
        20   Yogurt 20    Dairy  43.14            144
        24   Butter 24    Dairy  19.89            355
        25    Cream 25    Dairy   5.79            349
        29   Cheese 29    Dairy  20.16             59
        58     Milk 58    Dairy  24.14            110
        61   Butter 61    Dairy  12.56            407
        82   Cheese 82    Dairy  41.08            242
        84   Butter 84    Dairy  26.03            146
        85    Cream 85    Dairy  24.76            234
        95   Cheese 95    Dairy  48.85            229
       1

C:\Users\USER\AppData\Local\Temp\ipykernel_7464\385724960.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_dairy = pd.read_sql(query_dairy, conn)
C:\Users\USER\AppData\Local\Temp\ipykernel_7464\385724960.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_low_stock = pd.read_sql(query_low_stock, conn)
